In [ ]:
import pymc3 as pm
import pandas as pd
import numpy as np
# Dependency to be removed
from poibin import PoiBin
import pendulum as pend

In [ ]:
# This is something I have readily available from my code.
# It will be much simpler for actual workshops
def get_goal_ps(ps):
    pb = PoiBin(ps)
    goals = []
    probs = []
    g = 0
    p = 1
    while ((p > 0.01) or (g <= 3)) and (g <= ps.shape[0]):
        p = pb.pmf(g)
        goals.append(g)
        probs.append(p)
        g += 1
    probs = probs[:-1]
    goals = goals[:-1]
    return probs, goals

dt = pd.read_csv("https://git.io/fNmRy")
dt = dt.loc[dt.season == 2017]
# For each match, collect the expected goals for each team
match_id, time, home_team, i_home, away_team, i_away, goal_home, goal_away, weight = [], [], [], [], [], [], [], [], []
# Create a parallel data with only results
match_id_only, time_only, home_team_only, i_home_only, away_team_only, i_away_only, goal_home_only, goal_away_only = [], [], [], [], [], [], [], []
# Create a dictionary of teams for the i_home/i_away column
teams = dt.home.unique()
team_dic = {team:i for i, team in enumerate(teams)}


matches = dt.groupby('match_id')
# Construct the weighted and non-weighted data
for i, match in enumerate(matches):
    m_id = match[0]
    # print(m_id)
    match = match[1]
    # Get the time in unix timestamp
    time_ = pend.parse(match.date.iloc[0])
    time_ = time_.int_timestamp
    ps = match.loc[match.side=='h', 'xg'].to_numpy()
    p_home, g_home = get_goal_ps(ps)
    ps = match.loc[match.side=='a', 'xg'].to_numpy()
    p_away, g_away = get_goal_ps(ps)
    for (g_h, p_h), (g_a, p_a) in product(zip(g_home, p_home), zip(g_away, p_away)):
        match_id.append(m_id)
        goal_home.append(g_h)
        goal_away.append(g_a)
        # We should have probabilities high enough for this not to be a problem
        weight.append(p_a * p_h)
        home_team.append(match.iloc[0, 2])
        i_home.append(team_dic[match.iloc[0, 2]])
        away_team.append(match.iloc[0, 3])
        i_away.append(team_dic[match.iloc[0, 3]])
        time.append(time_)
    match_id_only.append(m_id)
    time_only.append(time_)
    home_team_only.append(match.iloc[0, 2])
    away_team_only.append(match.iloc[0, 3])
    i_home_only.append(team_dic[match.iloc[0, 2]])
    i_away_only.append(team_dic[match.iloc[0, 3]])
    goal_home_only.append(match.iloc[0,4])
    goal_away_only.append(match.iloc[0,5])

dt_simple = pd.DataFrame.from_dict({'match_id':match_id_only, 'time':time_only, 'home_team':home_team_only,
                                    'away_team':away_team_only, 'goal_home': goal_home_only, 'goal_away':goal_away_only,
                                    'i_home':i_home_only, 'i_away': i_away_only})



### Overall Objective

We just read the data for the 2017/2018 English Premier League (EPL) season. The title decider for this season was the match between Man City and Liverpool on ... . Our final objective for this workshop is to create a logistic regression model to predict who will win the match -- and probably the season!


To do this, we will first estimate a linear model of how many goals each team score. We will then proceed with a logistic regression predicting the probability that Man City wins. As a background information: each team in EPL play against all other team twice a year. One of the two matches is played "at home", the other match is played "away from home".

### First Concept

Let's run our first model. We will try to model the goal scored by any team as a simple normal distribution. We start with a "weakly informative" prior: a normal with mean 0 and sigma 10. We specify the model with a `pm.Deterministic` variable. This is a special kind of variables we will cover more extensively below. For now, it is just a way to signal to `PyMC3` that we actually observe data for that variable.

Statistical note. This is a special kind of model: we input a normal distribution (prior) we get an updated normal distribution (posterior). This is the property of conjugacy. These models are helpful because we know their posterior just by math, so we can check our random sampling is working as expected.

In [ ]:
# create a model environment through the with syntax
with pm.Model() as normal_conjugate:

    # define priors, weakly informative Normal
    b0 = pm.Normal("b0", mu=0, sigma=10)
    # define the observed part of the model
    y  = pm.Deterministic('goals', b0, observed = dt_simple['goal_away'])
    

In [ ]:
# Change the prior of the distribution: from mu=0, sigma=10 to mu=1.5 sigma=0.5
with pm.Model() as normal_conjugate:
    # How do you change this?
    b0 = pm.Normal("b0", mu=0, sigma=10)
    y  = pm.Deterministic('goals', b0, observed = dt_simple['goal_away'])

# Plot the DAG of the model [need to check this is easy to do cross-platform]

### Second Concept

As you can guess, the ability of teams to score are widely different. Having the same normal distribution for everything is not great modeling. Let's single out our two teams and model their scoring with a simple linear regression. 

Statistical note. We subset the matches up to the big match because we use all available information up to that point. Moreover, we exclude the first game between Man City and Liverpool to preserve independence in the data.

In [ ]:
## Part where I subset the data. perhaps I can even do this offline
### Pandas syntax to get the right column 
obs_goal = dt_simple['goals']
team_var = dt_simple['team_var']
home_var = dt_simple['home_var']

Now we specify the linear model:

$Goal = \beta_0 + \beta_1 team$

where $team$ is a simple indicator whether the team scoring the goal is Manchester city ($team = 1$) or Liverpool ($team = 0$). This is stored in the `team_var` variable.

In [ ]:
# create a model environment again
with pm.Model() as linear_model:
    
    # PRIOR PART
    # This is the prior for the intercept
    b0 = pm.Normal("b0", mu=0, sigma=10)
    # This is the prior for the coefficient beta_1 (see equation above)
    b1 = pm.Normal("b0", mu=0, sigma=10)
    
    # LIKELIHOOD and OBSERVATION
    y  = pm.Deterministic('goals', b0 + b1*team_var, observed = obs_goal)

# PLOT THE MODEL DAG
pm.model_to_graphviz(model)

#### Exercise

You can see the linear model is an expansion of the normal model above (see Concept 1). Can you describe the difference in terms of coding?

Modify the code above to specify this model:

$Goal = \beta_0 + \beta_1 team + \beta_2 home$

where $home$ is a simple indicator whether the team considered is playing at home ($home = 1$) or away ($home = 0$). This is stored in the `home_var` variable.

In [ ]:
# MODIFY THE CODE
with pm.Model() as linear_model:
    
    # PRIOR PART
    # This is the prior for the intercept
    b0 = pm.Normal("b0", mu=0, sigma=10)
    # This is the prior for the coefficient beta_1
    b1 = pm.Normal("b0", mu=0, sigma=10)
    
    # LIKELIHOOD and OBSERVATION
    y  = pm.Deterministic('goals', b0 + b1*team_var, observed = obs_goal)

# PLOT THE MODEL DAG
pm.model_to_graphviz(model)

### Third Concept

You can use the previous model to calculate the probability that number of goal scored by Liverpool is higher than the goals scored by Man City, or viceversa -- we will come back to this below.

But perhaps it is just easier make the model a logistic model of the probability of Man City winning. This requires just two extra steps: the specification of a logistic function as a link function and the use of the probability in a binomial distribution:

$\mathcal{L} = \beta_0 + \beta_1 team + \beta_2 home\\
\pi = logit(\mathcal{L})\\
Win \sim Categorical(\pi) $

$\mathcal{L}$ is the linear part of the model, $logit$ is the link function in GLM parlance (the function that transforms your linear variable into a probability between 0 and 1), $Win$ is a Categorical variable equal to $1$ if the team won.

#### Exercise

As we did above, let's consider the home-away factor for our prediction.

In [ ]:
# MODIFY THE CODE: ADD THE home_var as predictor in the linear part of the model
with pm.Model() as linear_model:
    
    # PRIOR PART
    # This is the prior for the intercept
    b0 = pm.Normal("b0", mu=0, sigma=10)
    # This is the prior for the coefficient beta_1
    b1 = pm.Normal("b0", mu=0, sigma=10)
    
    # LIKELIHOOD and OBSERVATION
    # Specify L
    L  = b0 + b1*team_var
    # Specify Pi
    pi = pm.Deterministic('goals', pm.math.invlogit(L))
    # Observed outcome (did Man City win?)
    L  = pm.Categorical('win', pi, observed = win)

# PLOT THE MODEL DAG
pm.model_to_graphviz(model)